In [1]:
from datasets import load_dataset, Dataset
import evaluate
import numpy as np
import os
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    BitsAndBytesConfig,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)

/app/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

In [3]:
mixed_sample = pd.read_csv("emails_bootstrapped_mixed_sample.csv")

In [4]:
mixed_sample["bootstrap_spam"] = mixed_sample["spam"]

In [5]:
def write_jsonl(df, filename: str):
    if os.path.isfile(filename):
        os.remove(filename)
        pass
    with open(filename, "a") as f:
        for _, row in df.iterrows():
            f.write(row.to_json())
            f.write("\n")
            pass
        pass
    return

In [6]:
write_jsonl(
    mixed_sample.rename(
        columns={"spam": "labels"}
    )[["text", "labels"]],
    "email_train_0.jsonl",
)

In [7]:
model_id = "microsoft/deberta-v3-base"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/app/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/app/venv/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [8]:
LABELS = {0: 0, 1: 1}

In [9]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    device_map={"": torch.cuda.current_device()},
    num_labels=2,
    trust_remote_code=True,
    id2label=LABELS, 
    label2id=LABELS,
)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
print(tokenizer.model_max_length)
print(model.config.max_position_embeddings)
tokenizer.model_max_length = model.config.max_position_embeddings

1000000000000000019884624838656
512


In [11]:
def tokenize_dataset(text_ds: Dataset):
    tokenized = text_ds.map(
        lambda xs: tokenizer(
            xs["text"], 
            truncation=True,
            padding="max_length",
        ),
        batched=True,
        remove_columns=["text"],
    )
    tokenized.set_format("torch")
    return tokenized

In [12]:
dataset = load_dataset(
    "json",
    data_files="email_train_0.jsonl",
)["train"].train_test_split(test_size=0.2)
tokenized_dataset = tokenize_dataset(dataset)

Generating train split: 100 examples [00:00, 10788.65 examples/s]
Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 697.71 examples/s]


In [13]:
from transformers import get_scheduler
from tqdm.auto import tqdm
from torch.optim import AdamW
from torch.utils.data import DataLoader

num_epochs = 10
batch_size = 8
num_training_steps = num_epochs * len(tokenized_dataset["train"]) // batch_size
loss_report_interval = num_training_steps // 10
device = "cuda"

train_dataloader = DataLoader(
    tokenized_dataset["train"], 
    batch_size=batch_size,
)
test_dataloader = DataLoader(
    tokenized_dataset["test"], 
    batch_size=batch_size,
)
optimizer = AdamW(model.parameters(), lr=5e-5)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [14]:
progress_bar = tqdm(range(num_training_steps))
model.train()
for epoch in range(num_epochs):
    for i, batch in enumerate(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        if i % loss_report_interval == 0:
            print("loss:", loss.mean(dim=0).item())
            pass
        del outputs
        del loss
        torch.cuda.empty_cache()
        #print("using:", torch.cuda.memory_allocated() / 1e6, "MB")

  1%|█▋                                                                                                                                                                          | 1/100 [00:00<00:57,  1.73it/s]

loss: 0.6486430764198303


 11%|██████████████████▊                                                                                                                                                        | 11/100 [00:05<00:41,  2.16it/s]

loss: 0.5212352871894836


 21%|███████████████████████████████████▉                                                                                                                                       | 21/100 [00:09<00:36,  2.15it/s]

...

 81%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                | 81/100 [00:37<00:08,  2.15it/s]

loss: 0.0031561374198645353


 91%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌               | 91/100 [00:42<00:04,  2.15it/s]

loss: 0.0032790552359074354


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:46<00:00,  2.14it/s]

In [15]:
# evalute
metric = evaluate.load("accuracy")
model.eval()
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.9}

We can save the model to the hub to track our progress

In [16]:
model.push_to_hub("tengelisconsulting/email_classifier_deberta3", commit_message="v1")
tokenizer.push_to_hub("tengelisconsulting/email_classifier_deberta3", commit_message="v1")


model.safetensors:   0%|                                                                                                                                                              | 0.00/738M [00:00<?, ?B/s]
model.safetensors:   1%|█▍                                                                                                                                                   | 7.18M/738M [00:00<00:10, 71.7MB/s]
model.safetensors:   2%|███▏                                                                                                                                                 | 16.0M/738M [00:00<00:25, 28.0MB/s]
model.safetensors:   4%|██████▍                                                                                                                                              | 32.0M/738M [00:00<00:17, 39.7MB/s]
model.safetensors:   6%|█████████▎                                                                                                                             

CommitInfo(commit_url='https://huggingface.co/tengelisconsulting/email_classifier_deberta3/commit/e9eb5ccb2094a18123b1f759d7ab179c902a3f0f', commit_message='v1', commit_description='', oid='e9eb5ccb2094a18123b1f759d7ab179c902a3f0f', pr_url=None, pr_revision=None, pr_num=None)

# Now we make predictions about the entire dataset, and compare with our initial bootstrapping

In [17]:
df = pd.read_csv("./emails_bootstrapped.csv")
df

,text,spam,bootstrap_spam
0,Subject: naturally irresistible your corporate...,1,1
1,Subject: the stock trading gunslinger fanny i...,1,1
2,Subject: unbelievable new homes made easy im ...,1,1
3,Subject: 4 color printing special request add...,1,1
4,"Subject: do not have money , get software cds ...",1,1
...,...,...,...
5723,Subject: re : research and development charges...,0,0
5724,"Subject: re : receipts from visit jim , than...",0,0
5725,Subject: re : enron case study update wow ! a...,0,0
5726,"Subject: re : interest david , please , call...",0,1


In [18]:
dl = DataLoader(
    tokenize_dataset(
        Dataset.from_pandas(df[["text"]])
    ), 
    batch_size=16,
)


Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5728/5728 [00:02<00:00, 2257.80 examples/s]


In [20]:
df["spam_v1"] = -1

results = []
progress_bar = tqdm(range(len(dl)))
model.eval()
for batch in dl:
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model(**batch)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    results.extend(predictions.tolist())
    progress_bar.update(1)
    pass

df["spam_v1"] = results


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:52<00:00,  1.13s/it]

 98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏  | 352/358 [00:05<00:00, 69.69it/s]
368it [00:05, 69.75it/s]                                                                                                                                                                                         
384it [00:05, 69.75it/s]
400it [00:05, 69.75it/s]
416it [00:05, 69.65it/s]
432it [00:06, 69.69it/s]
448it [00:06, 69.71it/s]
464it [00:06, 69.67it/s]
480it [00:06, 69.64it/s]
496it [00:07, 69.64it/s]
512it [00:07, 69.71it/s]
528it [00:07, 69.63it/s]
544it [00:07, 69.52it/s]
560it [00:08, 69.10it/s]
576it [00:08, 69.21it/s]
592it [00:08, 69.23it/s]
608it [00:08, 69.2

In [21]:
len(df[df["spam"] == df["spam_v1"]]) / len(df)

0.8933310055865922

Wow, that's already pretty good!  Especially because this classifier is so small.
To improve it, we just need some more (verified) samples.
We'll proceed with verifying the predictions where we have disagreed with the initial 'bootstrapped' prediction:

In [23]:
# first, better start labelling which rows we have 'verified'
df["verified"] = False
df.loc[0:49, "verified"] = True  # 'loc' is inclusive on the end index
df.loc[len(df)-50:len(df), "verified"] = True
df[df["verified"]]

,text,spam,bootstrap_spam,spam_v1,verified
0,Subject: naturally irresistible your corporate...,1,1,1,True
1,Subject: the stock trading gunslinger fanny i...,1,1,1,True
2,Subject: unbelievable new homes made easy im ...,1,1,1,True
3,Subject: 4 color printing special request add...,1,1,1,True
4,"Subject: do not have money , get software cds ...",1,1,1,True
...,...,...,...,...,...
5723,Subject: re : research and development charges...,0,0,0,True
5724,"Subject: re : receipts from visit jim , than...",0,0,0,True
5725,Subject: re : enron case study update wow ! a...,0,0,0,True
5726,"Subject: re : interest david , please , call...",0,1,0,True


In [24]:
mismatches = df[(df["bootstrap_spam"] != df["spam_v1"]) & ~df["verified"]]
mismatches

,text,spam,bootstrap_spam,spam_v1,verified
55,Subject: letter from : daniel kabila letter f...,1,1,0,False
60,Subject: partnership mr . edward moko 18 ind...,1,1,0,False
80,Subject: update your account information dear...,1,0,1,False
86,Subject: 00971 50 2443308 kevin contact me he...,1,1,0,False
90,Subject: investment / partnership proposal de...,1,1,0,False
...,...,...,...,...,...
5656,"Subject: new eprm speakers vince , thanks ve...",0,1,0,False
5659,"Subject: rtp project john and krishna , i am...",0,1,0,False
5664,Subject: june 21 - 22 retail electricity confe...,0,1,0,False
5667,"Subject: re : info help . krishna , niclas i...",0,1,0,False


Again, let's verify 50 spam and 50 non-spam emails

In [26]:
not_spam_idx = mismatches[mismatches["bootstrap_spam"] == 0].iloc[0:50].index
spam_idx = mismatches[mismatches["bootstrap_spam"] == 1].iloc[0:50].index

In [27]:
df.loc[not_spam_idx, "verified"] = True
df.loc[spam_idx, "verified"] = True

Now we pretty much just repeat this process over and over...

In [29]:
def write_verified_rows(version: int):
    rows = df[df["verified"]]
    write_jsonl(
        rows.rename(
            columns={"spam": "labels"}
        )[["text", "labels"]],
        f"email_train_v{version}.jsonl",
    )
    return

In [30]:
def get_dataloaders(*, version: int, batch_size: int):
    dataset = load_dataset(
        "json",
        data_files=f"email_train_v{version}.jsonl",
    )["train"].train_test_split(test_size=0.2)
    tokenized_dataset = tokenize_dataset(dataset)    
    train_dataloader = DataLoader(
        tokenized_dataset["train"], 
        batch_size=batch_size,
        shuffle=True,
    )
    test_dataloader = DataLoader(
        tokenized_dataset["test"], 
        batch_size=batch_size,
        shuffle=True,
    )
    return train_dataloader, test_dataloader

In [31]:
def train(
        *,
        version: int,
        num_epochs: int,
        batch_size: int,
        learning_rate: float=5e-5,
):
    device = "cuda"
    write_verified_rows(version)
    train_dl, _test_dl = get_dataloaders(version=version, batch_size=batch_size)    
    num_training_steps = num_epochs * len(train_dl) // batch_size
    loss_report_interval = num_training_steps // 10

    optimizer = AdamW(model.parameters(), lr=learning_rate)
    lr_scheduler = get_scheduler(
        name="linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps,
    )

    progress_bar = tqdm(range(num_training_steps))
    model.train()
    for epoch in range(num_epochs):
        for i, batch in enumerate(train_dataloader):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)
            if i % loss_report_interval == 0:
                print("loss:", loss.mean(dim=0).item())
                pass
            del outputs
            del loss
            torch.cuda.empty_cache()
            #print("using:", torch.cuda.memory_allocated() / 1e6, "MB")
            pass
        pass
    return

In [32]:
def evalute(
        *,
        version: int,
        batch_size: int,
):
    metric = evaluate.load("accuracy")
    _train_dl, test_dl = get_dataloaders(
        version=version,
        batch_size=batch_size,
    )
    model.eval()
    for batch in test_dl:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch["labels"])

    return metric.compute()


In [50]:
def make_predictions(
        *,
        version: int,
        batch_size: int,
):
    model.eval()
    df[f"spam_v{version}"] = -1
    dl = DataLoader(
        tokenize_dataset(
            Dataset.from_pandas(df[["text"]])
        ),
        batch_size=batch_size,
    )
    results = []
    progress_bar = tqdm(range(len(dl)))
    model.eval()
    for batch in dl:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        results.extend(predictions.tolist())
        progress_bar.update(1)
        pass

    df[f"spam_v{version}"] = results


In [34]:
def mock_verify_new_rows(
        *,
        version: int,
        n: int = 100,
):
    mismatches = df[(df["bootstrap_spam"] != df[f"spam_v{version}"]) & ~df["verified"]].iloc[:n]
    # we prefer mismatches, but we'll just grab any unferified rows if need be
    verify_indexes = mismatches.index
    if len(mismatches) < n:
        n_extra = n - len(mismatches)
        extra = df[
            (df["bootstrap_spam"] == df[f"spam_v{version}"]) & ~df["verified"]
        ].iloc[:n_extra]
        verify_indexes = verify_indexes.union(extra.index)
        pass
    df.loc[verify_indexes, "verified"] = True
    return verify_indexes


In [41]:
# (if you need to restart the kernel, and pick up where you left off training)
model = AutoModelForSequenceClassification.from_pretrained(
    "tengelisconsulting/email_classifier_deberta3",
    device_map={"": torch.cuda.current_device()},
    num_labels=2,
    trust_remote_code=True,
    id2label=LABELS, 
    label2id=LABELS,
)

In [55]:
VERSION = 4

In [56]:
train(
    version=VERSION,
    num_epochs=10,
    batch_size=8,
)



Generating train split: 400 examples [00:00, 24867.29 examples/s]


Map:   0%|                                                                                                                                                                        | 0/320 [00:00<?, ? examples/s]

Map: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 320/320 [00:00<00:00, 1726.86 examples/s]


Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 1453.45 examples/s]


  0%|                                                                                                                                                                                     | 0/50 [00:00<?, ?it/s]

  2%|███▍                                                                           

loss: 1.0430812125150624e-07




  4%|██████▉                                                                                                                                                                      | 2/50 [00:00<00:18,  2.55it/s]

  6%|██████████▍                                                                                                                                                                  | 3/50 [00:01<00:19,  2.45it/s]

  8%|█████████████▊                                                                                                                                                               | 4/50 [00:01<00:18,  2.43it/s]

 10%|█████████████████▎                                                                                                                                                           | 5/50 [00:02<00:18,  2.42it/s]

 12%|████████████████████▊                                                                                                                                

loss: 1.4901160305669237e-08




 14%|████████████████████████▏                                                                                                                                                    | 7/50 [00:02<00:17,  2.40it/s]

 16%|███████████████████████████▋                                                                                                                                                 | 8/50 [00:03<00:17,  2.40it/s]

 18%|███████████████████████████████▏                                                                                                                                             | 9/50 [00:03<00:17,  2.40it/s]

 20%|██████████████████████████████████▍                                                                                                                                         | 10/50 [00:04<00:16,  2.40it/s]

 22%|█████████████████████████████████████▊                                                                                                               

...



87it [00:36,  2.41it/s]

88it [00:36,  2.41it/s]

89it [00:37,  2.41it/s]

90it [00:37,  2.41it/s]

91it [00:37,  2.41it/s]

loss: 0.0




92it [00:38,  2.40it/s]

93it [00:38,  2.40it/s]

94it [00:39,  2.40it/s]

95it [00:39,  2.40it/s]

96it [00:40,  2.40it/s]

loss: 0.0




97it [00:40,  2.40it/s]

98it [00:40,  2.40it/s]

99it [00:41,  2.41it/s]

100it [00:41,  2.39it/s]


In [57]:
evalute(version=VERSION, batch_size=8)



Map:   0%|                                                                                                                                                                        | 0/320 [00:00<?, ? examples/s]

Map: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 320/320 [00:00<00:00, 2064.98 examples/s]


Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 1577.63 examples/s]


{'accuracy': 0.65}

In [58]:
make_predictions(version=VERSION, batch_size=8)



Map:   0%|                                                                                                                                                                       | 0/5728 [00:00<?, ? examples/s]

Map:  17%|██████████████████████████▉                                                                                                                               | 1000/5728 [00:00<00:01, 3391.26 examples/s]

Map:  35%|█████████████████████████████████████████████████████▊                                                                                                    | 2000/5728 [00:00<00:01, 3037.00 examples/s]

Map:  52%|████████████████████████████████████████████████████████████████████████████████▋                                                                         | 3000/5728 [00:01<00:00, 2938.48 examples/s]

Map:  70%|███████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                    

In [59]:
len(df[df[f"spam_v{VERSION}"] == df["spam"]]) / len(df)

0.9561801675977654

In [60]:
mock_verify_new_rows(version=VERSION)

Index([1653, 1681, 1682, 1700, 1705, 1730, 1808, 1818, 1834, 1848, 1878, 1883,
       1884, 1885, 1896, 1901, 1910, 1911, 1929, 1936, 1937, 1940, 1947, 1949,
       1953, 1954, 1956, 1957, 1970, 1975, 1984, 1991, 1994, 1999, 2003, 2011,
       2012, 2015, 2016, 2025, 2027, 2028, 2034, 2035, 2048, 2050, 2051, 2054,
       2060, 2061, 2072, 2073, 2074, 2078, 2079, 2097, 2098, 2104, 2105, 2114,
       2117, 2119, 2121, 2138, 2143, 2145, 2154, 2157, 2160, 2179, 2182, 2183,
       2184, 2187, 2189, 2192, 2193, 2196, 2202, 2203, 2204, 2206, 2207, 2213,
       2219, 2220, 2223, 2232, 2238, 2244, 2247, 2250, 2251, 2254, 2255, 2262,
       2263, 2265, 2269, 2271],
      dtype='int64')

In [61]:
df[df["verified"]]

,text,spam,bootstrap_spam,spam_v1,verified,spam_v2,spam_v3,spam_v4
0,Subject: naturally irresistible your corporate...,1,1,1,True,1,1,1
1,Subject: the stock trading gunslinger fanny i...,1,1,1,True,1,1,1
2,Subject: unbelievable new homes made easy im ...,1,1,1,True,1,1,1
3,Subject: 4 color printing special request add...,1,1,1,True,1,1,1
4,"Subject: do not have money , get software cds ...",1,1,1,True,1,1,1
...,...,...,...,...,...,...,...,...
5723,Subject: re : research and development charges...,0,0,0,True,0,0,0
5724,"Subject: re : receipts from visit jim , than...",0,0,0,True,0,0,0
5725,Subject: re : enron case study update wow ! a...,0,0,0,True,0,0,0
5726,"Subject: re : interest david , please , call...",0,1,0,True,0,0,0


In [62]:
model.push_to_hub("tengelisconsulting/email_classifier_deberta3", commit_message=f"v{VERSION}")
tokenizer.push_to_hub("tengelisconsulting/email_classifier_deberta3", commit_message=f"v{VERSION}")



model.safetensors:   0%|                                                                                                                                                              | 0.00/738M [00:00<?, ?B/s]

model.safetensors:   0%|▋                                                                                                                                                    | 3.24M/738M [00:00<00:23, 31.3MB/s]

model.safetensors:   2%|███▏                                                                                                                                                 | 16.0M/738M [00:00<00:22, 32.2MB/s]

model.safetensors:   4%|██████▍                                                                                                                                              | 32.0M/738M [00:00<00:18, 38.9MB/s]

model.safetensors:   6%|████████▉                                                                                                                         

CommitInfo(commit_url='https://huggingface.co/tengelisconsulting/email_classifier_deberta3/commit/0e207313a154d3a2843b415c252ba0186205cdf0', commit_message='v4', commit_description='', oid='0e207313a154d3a2843b415c252ba0186205cdf0', pr_url=None, pr_revision=None, pr_num=None)